In [2]:
import numpy as np
from qutip import basis, ket2dm, tensor, expect, fidelity, qeye

# Define the states
H = basis(2,0)
V = basis(2,1)

## Maximally entangled EPR pairs:
phi_plus_ket = (tensor(H, H) + tensor(V, V)).unit()
phi_plus = ket2dm(phi_plus_ket)

## Maximally entangled EPR pairs with mixture:
rho = 0.7 * phi_plus + 0.15 * ket2dm(tensor(H, V)) + 0.15 * ket2dm(tensor(V, H))


## Non-maximally entangled pair:
phi_ket = (1/np.sqrt(5)*tensor(H, H) + 2/np.sqrt(5)*tensor(V, V)).unit()
phi = ket2dm(phi_ket)


## Non-maximally entangled pair with mixture:
rho_prime = 0.7*phi + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

all_states = {
    "Phi+": phi_plus,
    "rho": rho,
    "phi": phi,
    "rho_prime": rho_prime
}

### Generate entangled-pair timestamps (shared for Alice & Bob)

In [3]:
def generate_pair_timestamps(pair_rate, total_time):
    """
    Generate an array of 'pair' arrival times in [0, total_time],
    from a Poisson process with average rate = pair_rate.
    """
    n_pairs = np.random.poisson(pair_rate * total_time)
    return np.random.rand(n_pairs) * total_time

### Detect real photons on each side from the shared pair times

In [4]:
def detect_photons_from_pairs(pair_times, eff_detector, dead_time, total_time):
    """
    For each pair time in 'pair_times':
      - We attempt detection with probability eff_detector
      - We skip arrivals if the detector is 'dead' from a previous detection
        for 'dead_time' seconds.

    Returns: np.array of detection times (photons)
    """
    times_sorted = np.sort(pair_times)
    detect_times = []
    last_det_t = -np.inf
    for t in times_sorted:
        if t - last_det_t < dead_time:
            continue
        # attempt detection
        if np.random.rand() < eff_detector:
            detect_times.append(t)
            last_det_t = t
    return np.array(detect_times)

### Generate & detect dark counts

In [5]:
def generate_dark_counts(dark_rate, eff_detector, dead_time, total_time):
    """
    Simulate dark-count arrivals at 'dark_rate' (Poisson process),
    then apply detection logic (eff_detector + dead_time).
    """
    n_dark = np.random.poisson(dark_rate * total_time)
    dark_arrivals = np.sort(np.random.rand(n_dark) * total_time)

    detect_times = []
    last_det_t = -np.inf
    for t in dark_arrivals:
        if t - last_det_t < dead_time:
            continue
        # attempt detection
        if np.random.rand() < eff_detector:
            detect_times.append(t)
            last_det_t = t
    return np.array(detect_times)

### Counting coincidences with a 1 ns window, separating photon/dark

In [6]:
def count_coincidences(a_photon, a_dark, b_photon, b_dark, coinc_window=1e-9):
    """
    Return a dict with number of coincidences in each category:
      - photon-photon
      - photon-dark
      - dark-photon
      - dark-dark
    """
    cdict = {}
    cdict["photon-photon"] = _count_coincidences_2arr(a_photon, b_photon, coinc_window)
    cdict["photon-dark"]   = _count_coincidences_2arr(a_photon, b_dark, coinc_window)
    cdict["dark-photon"]   = _count_coincidences_2arr(a_dark, b_photon, coinc_window)
    cdict["dark-dark"]     = _count_coincidences_2arr(a_dark, b_dark, coinc_window)
    return cdict

def _count_coincidences_2arr(times1, times2, window):
    """
    Two-pointer approach to count how many (t1,t2) differ by <= window.
    """
    i, j = 0, 0
    n_coinc = 0
    while i < len(times1) and j < len(times2):
        dt = times1[i] - times2[j]
        if abs(dt) <= window:
            n_coinc += 1
            i += 1
            j += 1
        elif dt < 0:
            i += 1
        else:
            j += 1
    return n_coinc

### Compute CHSH S-value from density matrices

In [7]:
def projector(theta):
    """ Projector onto cos(theta)|H> + sin(theta)|V> for 1 qubit. """
    state = np.cos(theta)*H + np.sin(theta)*V
    return ket2dm(state.unit())

def compute_expectation(rho_2q, theta_a, theta_b):
    """
    E(a, b) = (Prob(++ or --) - Prob(+- or -+)).
    We'll define A_plus, B_plus, etc. and compute directly.
    """
    A_plus = projector(theta_a)
    B_plus = projector(theta_b)

    A_minus = qeye(2) - A_plus
    B_minus = qeye(2) - B_plus

    P_pp = tensor(A_plus, B_plus)         # ++
    P_mm = tensor(A_minus, B_minus)       # --
    P_pm = tensor(A_plus, B_minus)        # +-
    P_mp = tensor(A_minus, B_plus)        # -+

    p_pp = (P_pp * rho_2q).tr().real
    p_mm = (P_mm * rho_2q).tr().real
    p_pm = (P_pm * rho_2q).tr().real
    p_mp = (P_mp * rho_2q).tr().real

    return (p_pp + p_mm) - (p_pm + p_mp)



In [9]:
# Parameters
PAIR_RATE   = 15000    # entangled pairs per second
DARK_RATE   = 1000     # dark counts/s
EFF_DET     = 0.10     # 10% detection efficiency
DEAD_TIME   = 4e-6     # 4 microseconds
T_RUN       = 30       # run for 30 seconds
COINC_WIN   = 1e-9     # 1 ns

results = {}

for state_name, rho_2q in all_states.items():
    #--- (A) Generate entangled-pair arrival times
    pair_times = generate_pair_timestamps(PAIR_RATE, T_RUN)

    #--- (B) Local detection for real photons
    alice_photon_t = detect_photons_from_pairs(pair_times, EFF_DET, DEAD_TIME, T_RUN)
    bob_photon_t   = detect_photons_from_pairs(pair_times, EFF_DET, DEAD_TIME, T_RUN)

    #--- (C) Dark counts on each side
    alice_dark_t = generate_dark_counts(DARK_RATE, EFF_DET, DEAD_TIME, T_RUN)
    bob_dark_t   = generate_dark_counts(DARK_RATE, EFF_DET, DEAD_TIME, T_RUN)

    #--- (D) Merge times if you want total detections
    alice_all = np.sort(np.concatenate([alice_photon_t, alice_dark_t]))
    bob_all   = np.sort(np.concatenate([bob_photon_t, bob_dark_t]))

    #--- (E) Coincidence analysis with separate categories
    cdict = count_coincidences(alice_photon_t, alice_dark_t,
                                bob_photon_t, bob_dark_t,
                                coinc_window=COINC_WIN)
    # total coincidence (any detection)
    def _count_any_coinc(a_times, b_times):
        return _count_coincidences_2arr(a_times, b_times, COINC_WIN)

    total_coinc = _count_any_coinc(alice_all, bob_all)
    coincidence_rate = total_coinc / T_RUN

    #--- (F) Detection rates
    alice_rate = len(alice_all) / T_RUN
    bob_rate   = len(bob_all)   / T_RUN

    #--- (G) Fidelity vs. |Phi+>
    F = fidelity(rho_2q, phi_plus)

    #--- (H) Bell S value (theoretical from the DM)
    # Typical angles for CHSH:
    E1 = compute_expectation(rho_2q, 0,       np.pi/8)
    E2 = compute_expectation(rho_2q, 0,       3*np.pi/8)
    E3 = compute_expectation(rho_2q, np.pi/4, np.pi/8)
    E4 = compute_expectation(rho_2q, np.pi/4, 3*np.pi/8)
    S = E1 - E2 + E3 + E4

    results[state_name] = {
        "Alice total rate (counts/s)": alice_rate,
        "Bob total rate (counts/s)":   bob_rate,

        "Alice photon rate": len(alice_photon_t)/T_RUN,
        "Alice dark rate":   len(alice_dark_t)/T_RUN,
        "Bob photon rate":   len(bob_photon_t)/T_RUN,
        "Bob dark rate":     len(bob_dark_t)/T_RUN,

        "Coincidences (ph-ph)":   cdict["photon-photon"],
        "Coincidences (ph-dark)": cdict["photon-dark"],
        "Coincidences (dark-ph)": cdict["dark-photon"],
        "Coincidences (dark-dark)": cdict["dark-dark"],

        "Total coincidences":       total_coinc,
        "Coincidence rate (counts/s)": coincidence_rate,

        "Fidelity vs. PhiPlus": float(F),
        "Bell S value": float(S)
    }

# Print results
for st, val in results.items():
    print(f"=== State: {st} ===")
    for k, v in val.items():
        print(f"  {k}: {v}")
    print()

=== State: Phi+ ===
  Alice total rate (counts/s): 1581.5
  Bob total rate (counts/s): 1592.7
  Alice photon rate: 1482.4666666666667
  Alice dark rate: 99.03333333333333
  Bob photon rate: 1490.8666666666666
  Bob dark rate: 101.83333333333333
  Coincidences (ph-ph): 4368
  Coincidences (ph-dark): 0
  Coincidences (dark-ph): 0
  Coincidences (dark-dark): 0
  Total coincidences: 4368
  Coincidence rate (counts/s): 145.6
  Fidelity vs. PhiPlus: 0.9999999999999988
  Bell S value: 2.828427124746189

=== State: rho ===
  Alice total rate (counts/s): 1595.5666666666666
  Bob total rate (counts/s): 1611.7666666666667
  Alice photon rate: 1497.9
  Alice dark rate: 97.66666666666667
  Bob photon rate: 1510.7
  Bob dark rate: 101.06666666666666
  Coincidences (ph-ph): 4594
  Coincidences (ph-dark): 0
  Coincidences (dark-ph): 1
  Coincidences (dark-dark): 0
  Total coincidences: 4595
  Coincidence rate (counts/s): 153.16666666666666
  Fidelity vs. PhiPlus: 0.8366600265340758
  Bell S value: 1.5